In [1]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.strategies.strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from quantfreedom.simulate import sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
import requests

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [2]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=1000)

It took 00 mins and 00 seconds to download 1000 candles


In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
# backtest_settings = BacktestSettings(
#     divide_records_array_size_by=1,
#     gains_pct_filter=-90,
#     total_trade_filter=10,
#     upside_filter=-np.inf,
# )
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([9]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [4]:
strat_records_df = sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.File,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 4
Total combinations of settings to test: 24
Total candles: 1,000
Total candles to test: 24,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,25.0,5,20,-38.53,20.00,-0.0000,478.665,-385.2806,614.7194
1,0,1,19.0,4,15,-29.37,21.05,-0.0000,340.600,-293.7037,706.2963
2,0,2,38.0,5,33,-75.96,13.16,-0.9324,581.767,-759.5855,240.4145
3,0,3,21.0,3,18,-57.41,14.29,-0.0000,355.371,-574.0813,425.9187
4,1,0,25.0,5,20,-38.68,20.00,-0.0000,477.563,-386.7866,613.2134
5,1,1,21.0,4,17,-28.83,19.05,-0.0000,373.495,-288.2717,711.7283
6,1,2,38.0,5,33,-75.96,13.16,-0.9324,581.767,-759.5855,240.4145
7,1,3,21.0,3,18,-57.41,14.29,-0.0000,355.371,-574.0813,425.9187
8,2,0,47.0,6,41,-80.73,12.77,-0.0000,675.381,-807.2720,192.7280
9,2,1,32.0,4,28,-72.80,12.50,-0.9030,503.347,-728.0183,271.9817


In [5]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

In [6]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,0,0,100,1698242400000,2023-10-25 14:00:00,EntryFilled,1000.0000,940.7730,3569.715,59.227,...,3628.942,34483.3,NaN,0.105,3628.942,NaN,0.65,34260.1,1.8,35095.0
1,0,0,101,1698242700000,2023-10-25 14:05:00,EntryFilled,1000.0000,856.5460,8946.122,143.454,...,5519.861,34385.2,NaN,0.266,9148.803,NaN,0.54,34260.1,1.4,34939.6
2,0,0,102,1698243000000,2023-10-25 14:10:00,EntryFilled,1000.0000,797.6950,12662.849,202.305,...,3919.032,34410.1,NaN,0.380,13067.835,NaN,0.52,34260.1,1.5,34953.7
3,0,0,118,1698247800000,2023-10-25 15:30:00,TakeProfitFilled,1180.2415,1180.2415,NaN,NaN,...,NaN,NaN,34953.7,NaN,NaN,180.2415,NaN,NaN,NaN,NaN
4,0,0,120,1698248400000,2023-10-25 15:40:00,EntryFilled,1180.2415,1127.2270,2191.961,53.015,...,2244.976,34811.8,NaN,0.064,2244.976,NaN,1.38,34331.3,3.2,35939.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,0,0,938,1698493800000,2023-10-28 11:50:00,EntryFilled,675.6029,525.6490,10570.191,149.954,...,3949.872,34188.7,NaN,0.318,10863.759,NaN,0.38,34078.1,1.2,34627.1
88,0,0,949,1698497100000,2023-10-28 12:45:00,StopLossFilled,614.7194,614.7194,NaN,NaN,...,NaN,NaN,34078.1,NaN,NaN,-60.8835,NaN,NaN,NaN,NaN
89,0,0,950,1698497400000,2023-10-28 12:50:00,EntryFilled,614.7194,558.8190,4639.571,55.901,...,4695.472,33999.7,NaN,0.138,4695.472,NaN,0.20,33930.5,0.9,34301.3
90,0,0,951,1698497700000,2023-10-28 12:55:00,EntryFilled,614.7194,510.0410,8564.880,104.679,...,4029.988,34021.3,NaN,0.256,8725.460,NaN,0.22,33930.5,0.9,34328.0
